In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))

# Site list

In [3]:
columns_string = ['Name', 'Symbol', 'Market Cap', 'Fully Diluted Market Cap', 'Volume 24h', 'Volume / Market Cap', 
                  'Circulating Supply', 'Self Reported Circulating Supply', 'Max Supply', 'Total Supply']

In [4]:
def statsContainer(): # get data from container
    try: return driver.find_element(By.CLASS_NAME, 'statsContainer').text.replace('Volume\n24h', 'Volume 24h').split('\n')
    except: return 'NaN'

In [5]:
def ativa_matriz_vazia():  
    matriz_vazia = [['Name', '--'], ['Symbol', '--'], ['Price', '--'], ['Market Cap', '--'], 
                    ['Fully Diluted Market Cap', '--'], ['Volume 24h', '--'], ['Volume / Market Cap', '--'],
                    ['Circulating Supply', '--'], ['Max Supply', '--'], ['Total Supply', '--']]
    return matriz_vazia

In [6]:
def name_symbol(matriz_vazia):
    Name, Symbol = driver.find_element(By.CLASS_NAME, 'h1').text.split('\n')
    matriz_vazia[0][1] = Name
    matriz_vazia[1][1] = Symbol
    return matriz_vazia

In [7]:
def price(matriz_p):
    price = driver.find_element(By.CLASS_NAME, 'priceSection').text.split('\n')[1]
    if (price == 'ICO') or (price == 'Market data is untracked'): matriz_p[2][1] = '--'
    else: matriz_p[2][1] = price.replace('$','')
    return matriz_p

In [8]:
def matriz_de_colunas(z): # separa em matriz de dados com título
    i = 0 
    j = -1
    lista_externa = []
    for zn in z:
        if zn in columns_string: 
            j=j+1
            if j >= 1 : lista_externa.append(lista_interna)
            lista_interna = []
            lista_interna.append(zn)
        else: lista_interna.append(zn)
        i = i+1
    lista_externa.append(lista_interna) # captura o ultimo intervalo da lista
    return lista_externa

In [9]:
def site_stuff(site):
    matriz_vazia = ativa_matriz_vazia()
    driver.get(site)
    matriz_p = name_symbol(matriz_vazia)
    matriz_p = price(matriz_p)
    lista = statsContainer() # string com as palavras chaves
    try: 
        matriz_Container = matriz_de_colunas(lista) # matriz separando em colunas chaves
        matriz_p = encaixa_container(matriz_p, matriz_Container)
    except: matriz_Container = [] # site sem os dados. evita erros.
    return matriz_p

In [10]:
def encaixa_container(matriz_p, matriz_Container):
    if matriz_Container[0][0] == 'Market Cap':
        if matriz_Container[0][1] == '- -': matriz_p[3][1] = '--'
        else: matriz_p[3][1] = matriz_Container[0][1].replace('$','')
    if matriz_Container[1][0] == 'Fully Diluted Market Cap': matriz_p[4][1] = matriz_Container[1][1].replace('$','')
    if matriz_Container[2][0] == 'Volume 24h': matriz_p[5][1] = matriz_Container[2][1].replace('$','')
    if matriz_Container[3][0] == 'Volume / Market Cap': matriz_p[6][1] = matriz_Container[3][1]
    if (matriz_Container[4][0] == 'Circulating Supply') or (matriz_Container[4][0] == 'Self Reported Circulating Supply'):
        matriz_p[7][1] = matriz_Container[4][1].split(' ')[0]
    if matriz_Container[5][0] == 'Max Supply': matriz_p[8][1] = matriz_Container[5][1]
    if matriz_Container[6][0] == 'Total Supply': matriz_p[9][1] = matriz_Container[6][1]
    return matriz_p

In [11]:
sites = ['https://coinmarketcap.com/currencies/dinox/', 'https://coinmarketcap.com/currencies/step-hero-step/',
        'https://coinmarketcap.com/currencies/forest-knight/', 'https://coinmarketcap.com/currencies/everfight/',
        'https://coinmarketcap.com/currencies/kaka-nft-world/', 'https://coinmarketcap.com/currencies/radio-caca/',
        'https://coinmarketcap.com/currencies/bloktopia/', 'https://coinmarketcap.com/currencies/cyberdragon-gold/',
        'https://coinmarketcap.com/currencies/singularfarm/', 'https://coinmarketcap.com/currencies/jojo/',
        'https://coinmarketcap.com/currencies/velas/', 'https://coinmarketcap.com/currencies/dogami/',
        'https://coinmarketcap.com/currencies/floki-inu/', 'https://coinmarketcap.com/currencies/baby-doge-coin/',
        'https://coinmarketcap.com/currencies/dogezilla/', 'https://coinmarketcap.com/currencies/fegtoken/',
        'https://coinmarketcap.com/currencies/project-quantum/','https://coinmarketcap.com/currencies/star-ship-royal/',
        'https://coinmarketcap.com/currencies/yooshi/', 'https://coinmarketcap.com/currencies/wanaka-farm/',
        'https://coinmarketcap.com/currencies/realfevr/', 'https://coinmarketcap.com/currencies/age-of-cryptology/',
        'https://coinmarketcap.com/currencies/doragonland/', 'https://coinmarketcap.com/currencies/polkamon/',
        'https://coinmarketcap.com/currencies/9d-nft/', 'https://coinmarketcap.com/currencies/heroes-and-empires/',
        'https://coinmarketcap.com/currencies/metavpad/', 'https://coinmarketcap.com/currencies/rofi/',
        'https://coinmarketcap.com/currencies/widiland/', 'https://coinmarketcap.com/currencies/kommunitas/',
        'https://coinmarketcap.com/currencies/meerkat-shares/', 'https://coinmarketcap.com/currencies/defina-finance/',
        'https://coinmarketcap.com/currencies/bombcrypto/', 'https://coinmarketcap.com/currencies/wairere-token/',
        'https://coinmarketcap.com/currencies/herocatgamefi/', 'https://coinmarketcap.com/currencies/coinary-token/',
        'https://coinmarketcap.com/currencies/monsta-infinite/', 'https://coinmarketcap.com/currencies/plantvsundead/',
        'https://coinmarketcap.com/currencies/wso/', 'https://coinmarketcap.com/currencies/hodler-heroes-nft/',
        'https://coinmarketcap.com/currencies/last-survivor/', 'https://coinmarketcap.com/currencies/fwar-finance/',
        'https://coinmarketcap.com/currencies/pegasusdollar/', 'https://coinmarketcap.com/currencies/lion-scrub-finance/',
        'https://coinmarketcap.com/currencies/x-world-games/', 'https://coinmarketcap.com/currencies/cryptogodz/',
        'https://coinmarketcap.com/currencies/mm-finance/', 'https://coinmarketcap.com/currencies/mad-meerkat-etf/',
        'https://coinmarketcap.com/currencies/savanna/', 'https://coinmarketcap.com/currencies/moniwar/',
        'https://coinmarketcap.com/currencies/plato-farm-plato/', 'https://coinmarketcap.com/currencies/legion-network/',
        'https://coinmarketcap.com/currencies/taroverse/', 'https://coinmarketcap.com/currencies/creo-engine/',
        'https://coinmarketcap.com/currencies/zone-of-avoidance/', 'https://coinmarketcap.com/currencies/decentraworld/']

In [12]:
df = pd.DataFrame(columns=columns_string)

In [13]:
for site in sites:
    x = site_stuff(site)
    lista_para_df = []
    for i in range (len(x)): lista_para_df.append(x[i][1])
    df.loc[len(df)] = lista_para_df

In [14]:
df

,Name,Symbol,Market Cap,Fully Diluted Market Cap,Volume 24h,Volume / Market Cap,Circulating Supply,Self Reported Circulating Supply,Max Supply,Total Supply
0,DinoX,DNXC,0.01594,"542,202","2,550,709","60,914",0.1123,"34,011,080.00","160,000,000","160,000,000"
1,Step Hero Soul,STEP,0.000696,"1,670","2,219",- -,--,"2,400,000.00",--,"3,188,252"
2,Forest Knight,KNIGHT,0.03125,"241,917","3,125,212","166,667",0.6889,"7,740,822.83","100,000,000","100,000,000"
3,EverFight,EF,--,--,--,--,--,--,--,--
4,KAKA NFT World,KAKA,0.02163,"119,072","2,163,057","42,138",0.3539,"5,504,800.00","100,000,000","100,000,000"
5,RadioCaca,RACA,0.0003723,"122,035,485","186,125,979","10,098,302",0.08275,327.83B,"500,000,000,000","431,418,233,271"
6,Bloktopia,BLOK,0.003329,"58,383,645","665,409,395","2,645,911",0.04532,17.54B,"199,868,396,351","200,000,000,000"
7,CyberDragon Gold,GOLD,0.03102,--,"6,796,907","2,173,537",--,--,"219,084,531","1,785,482,245"
8,SingularFarm,SING,0.008529,"4,713","10,234",0,0.00000002,"552,663.00","1,200,000","45,887"
9,JOJO,JOJO,0.0000369,--,- -,98,--,--,--,--


In [15]:
driver.close()